In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

import sklearn
from sklearn.model_selection import train_test_split
from matplotlib.colors import LogNorm

# Neural network libraries
import torch
import torchvision
import torch.utils.data as data_utils
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [2]:
current_path=!pwd
a=str(current_path[0])
c=a.replace("group_2","")
data_path=c+"challenge_setup/data_and_information/galstar.csv"
print("Reading data from: ",data_path)
galstar = pd.read_csv(data_path)

('Reading data from: ', '/g/g15/nedwards/challenge_setup/data_and_information/galstar.csv')


In [3]:
galstar.shape

(31798, 5410)

In [4]:
X = galstar.iloc[:,1:]
Y=galstar["y"]
vals=[]
X = X.iloc[:,1:]

In [5]:
count=0
count1=1
for i in Y:
    if(i==0):
        count=count+1
    if(i==1):
        count1=count1+1
    

In [6]:
#PIE chart of y counts
print("count of stars : "+str(count)+" count of Galaxiys : "+str(count1))
#visualize

count of stars : 18665 count of Galaxiys : 13134


In [7]:
vals=[]
for j in np.arange(31798):
    vals_temp=[]
    for i in np.arange(8):
        vals_temp.append(np.reshape(X.iloc[j].values[int(5408/8)*i:int(5408/8)*(i+1)],(26,26)))
    vals.append(vals_temp)
    if(j%100==0):
        print(j)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [8]:
np.shape(vals)
X=vals

In [9]:
#Declare Data
class Astroid(Dataset):
    def __init__(self,X,Y):
        self.X1=X
        self.Y1=Y
    def __getitem__(self, idx):
        idx=idx.item()
        tempX=self.X1[idx]
        tempX=torch.tensor(tempX)
        tempX=tempX.double()
        tempY=self.Y1[idx]
        if(tempY==1):
            tempY=torch.tensor([0,1])  
        else:
            tempY=torch.tensor([1,0])
            #(.2,.8)-[0]
            #(.2,.8)[0 1] [1 0]
            #
            #
        tempY=tempY.double()
        return(tempX,tempY)
       #returns batch(x,y)
    def __len__(self):
        return (len(self.X1))
        #Length




In [10]:
Data=Astroid(X,Y.values)


In [11]:
np.shape(X)

(31798, 8, 26, 26)

In [12]:
trainSize=22259
testSize=9539

In [13]:
trainSize+testSize

31798

In [14]:
train_dataset,val_dataset=torch.utils.data.random_split(Data,[int(trainSize),int(testSize)])


In [15]:
len(train_dataset)

22259

In [16]:
len(val_dataset)

9539

In [17]:
train_dataloader=DataLoader(train_dataset,batch_size=1,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=1)

In [267]:
class Astroid_Classifer(torch.nn.Module):
    """
    Simple model which takes 32x32 inputs and produces
    score values for each of 10 classes for each element.
    Model adapted from:
    https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
    """
    def __init__(self):
        super(Astroid_Classifer,self).__init__()
        # Features
        self.conv1=nn.Conv2d(8,16,3)
        self.pool1=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(16,32,3)
        self.conv3=nn.Conv2d(32,64,3)
        self.fc1=nn.Linear(64,48)#288/130
        self.fc2=nn.Linear(48,32)#130/80
        self.fc3=nn.Linear(32,16)#80/30
        self.fc4=nn.Linear(16,2)#30/2

    def forward(self, x):
        #Batch
        """
        Pass one batch of image data through the model.
        Return batch of score values corresponding to each class.
        Input size: (batch_size, 1, 32, 32)
        Output size: (batch_size, 10)
        """
        x=self.pool1(F.relu(self.conv1(x)))
        x=self.pool1(F.relu(self.conv2(x)))
        x=self.pool1(F.relu(self.conv3(x)))
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=(self.fc4(x))  
        return x

In [268]:
model = Astroid_Classifer()


In [269]:
accuracy=[]
loss_total=[]


In [285]:
criterion=nn.MSELoss()
optimzer=optim.Adagrad(model.parameters(),lr=.005)

In [286]:
for epoch in range(10):
    running_loss=float(0)
    count=0.0
    acc=0.0

    print("-------------TRAIN-------------")

    for i, (input_x, label) in enumerate(train_dataloader):
        optimzer.zero_grad()
        input_x=input_x.float()#Conversion x64 tensor to float
        outputs=model(input_x)#passes input tensor into model
        count=count+1
        label=label.float()
        loss=criterion(outputs,label)# [1,0],[1]
        
        if(torch.max(outputs, 1)[1]==torch.max(label, 1)[1]):
            acc=acc+1
            
        loss.backward()
        optimzer.step()
        running_loss+=loss.item()
        if i % 2000==1999:
            print('[%d,%5d] | loss: %.3f | accuracy: %.3f'%(epoch+1,i+1,running_loss/2000,acc/count))
            accuracy.append(acc/count)
            loss_total.append(running_loss/count)
            count=0.0
            acc=0.0
            running_loss=0

-------------TRAIN-------------
[1, 2000] | loss: 0.162 | accuracy: 0.779
[1, 4000] | loss: 0.127 | accuracy: 0.828
[1, 6000] | loss: 0.107 | accuracy: 0.857
[1, 8000] | loss: 0.093 | accuracy: 0.882
[1,10000] | loss: 0.091 | accuracy: 0.885
[1,12000] | loss: 0.082 | accuracy: 0.901
[1,14000] | loss: 0.089 | accuracy: 0.891
[1,16000] | loss: 0.078 | accuracy: 0.901
[1,18000] | loss: 0.076 | accuracy: 0.903
[1,20000] | loss: 0.066 | accuracy: 0.927
[1,22000] | loss: 0.075 | accuracy: 0.906
-------------TRAIN-------------
[2, 2000] | loss: 0.067 | accuracy: 0.917
[2, 4000] | loss: 0.063 | accuracy: 0.922
[2, 6000] | loss: 0.069 | accuracy: 0.920
[2, 8000] | loss: 0.066 | accuracy: 0.921
[2,10000] | loss: 0.059 | accuracy: 0.926
[2,12000] | loss: 0.064 | accuracy: 0.925
[2,14000] | loss: 0.070 | accuracy: 0.915
[2,16000] | loss: 0.061 | accuracy: 0.928
[2,18000] | loss: 0.065 | accuracy: 0.929
[2,20000] | loss: 0.061 | accuracy: 0.929
[2,22000] | loss: 0.058 | accuracy: 0.929
------------

In [298]:
path="Stars_and_Gal.pth.pth"

In [297]:
#weight=model.state_dict()
#torch.save(weight, path)

In [291]:
np.shape(accuracy)

(221,)

In [295]:
np.savetxt("loss.csv", loss_total, delimiter=",")


In [279]:
path="Stars_and_Gal.pth"

In [280]:
loaded_params=torch.load(path)
model.load_state_dict(loaded_params)

In [281]:
loaded_params

OrderedDict([('conv1.weight',
              tensor([[[[-4.1223e-03, -7.7201e-02, -1.1558e-01],
                        [ 2.1949e-02,  4.0410e-02,  1.8808e-02],
                        [-1.0402e-01,  6.4637e-02,  6.8546e-02]],
              
                       [[ 2.2622e-02,  2.2559e-02,  1.0506e-01],
                        [-9.8768e-02,  8.4024e-02, -4.6891e-02],
                        [-3.1909e-03, -7.2047e-02, -8.7380e-02]],
              
                       [[ 2.3805e-02, -2.6208e-02, -7.1921e-03],
                        [ 7.8343e-02, -5.3850e-02,  7.5675e-02],
                        [-2.6099e-02, -8.2156e-02,  5.0455e-02]],
              
                       ...,
              
                       [[-1.0975e-01,  5.2992e-02, -4.1387e-03],
                        [ 6.2066e-02,  1.8393e-02, -9.2548e-02],
                        [ 1.3569e-02, -1.0103e-01, -1.0790e-01]],
              
                       [[ 3.1626e-02, -9.8024e-02,  7.0745e-02],
                  

In [117]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
        for data in val_dataloader:
        input_val, labels = data
        # calculate outputs by running images through the network
        outputs = net(input_val)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test ima2ges: %d %%' % (
    100 * correct / total))


IndentationError: expected an indented block (<ipython-input-117-47e32cc3cb2a>, line 6)

In [ ]:
#Confusion matrix